#### Load Libraries and Demo Codes

In [1]:
import julia; julia.install(quiet=True)
from julia import Main

import numpy as np
import sympy as sp

import plotly.graph_objects as go
from scipy.spatial.transform import Rotation as R

import holoviews as hv
from holoviews import opts
import panel as pn
import param

pn.extension('katex', 'plotly')
hv.extension('bokeh')

%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [8]:
%%julia
using Pkg; Pkg.activate("../GenLinAlgProblems")
using GenLinAlgProblems, LinearAlgebra, Printf, Latexify, LaTeXStrings, Random, SymPy

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [9]:
#%output backend="bokeh"
class VectorAngleViewer(pn.viewable.Viewer):
    angle = param.Number(default=45, bounds=(0, 180), label="Angle (degrees)")

    def __init__(self, **params):
        super().__init__(**params)
        self.plot_pane = pn.pane.HoloViews(height=350)
        self.text_pane = pn.pane.LaTeX(width=250, renderer='katex')
        self._update_plot()
        self.param.watch(self._update_plot, ['angle'])
    def _update_plot(self, *_):
        angle_rad = np.radians(self.angle)
        u = np.array([1.0, 0.0])
        v = np.array([np.cos(angle_rad), np.sin(angle_rad)])
        dot_product = np.dot(u, v)

        segments = hv.Segments([
            (0, 0, u[0], u[1]),
            (0, 0, v[0], v[1])
        ]).opts(color='blue', line_width=3)

        arc_radius = 0.5
        angle_range = np.linspace(0, angle_rad, 100)
        arc_x = arc_radius * np.cos(angle_range)
        arc_y = arc_radius * np.sin(angle_range)
        arc = hv.Curve((arc_x, arc_y)).opts(color='darkred', line_width=2)#, line_dash='dotted')

        annotation = hv.Text(0.5, 1.6, f"Angle: {self.angle:.1f}°").opts(color='black')

        line = hv.HLine(0).opts(color='#989F7A', line_width=1.5)

        plot = (line * segments * arc * annotation).opts(
            width=400, height=400, xlim=(-1.5, 1.5), ylim=(0, 1.5),
            aspect='equal', title="Angle Between Two Vectors"
        )

        latex = self._latex_text(v[0], v[1], dot_product, self.angle)
        self.plot_pane.object = plot
        self.text_pane.object = latex
    def _latex_text(self, vx, vy, dot, angle):
        return (
            r"$\Large \begin{aligned}"
            r"u &= \begin{bmatrix} 1.00 \\ 0.00 \end{bmatrix} \\ "
            rf"v &= \begin{{bmatrix}} {vx:.2f} \\ {vy:.2f} \end{{bmatrix}} \\ "
            rf"u \cdot v &= {dot:.2f} \\ \\ \ "
            rf"\theta &= {angle:.2f}^\circ"
            r"\end{aligned}$"
        )
    def __panel__(self):
        return pn.Row(self.plot_pane, pn.Spacer(width=30),
                   pn.Column(pn.Spacer(height=20), self.text_pane,
                             pn.Spacer(height=20), pn.widgets.FloatSlider.from_param(self.param.angle, width=400)))

In [10]:
#%output backend="plotly"
def spherical_to_cartesian(r, theta_deg, phi_deg):
    theta = np.radians(theta_deg)
    phi = np.radians(phi_deg)
    return r * np.array([
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta)
    ])
def project_to_plane(v, normal):
    n = normal / np.linalg.norm(normal)
    return v - np.dot(v, n) * n
class VectorProjectionDemo(pn.viewable.Viewer):
    theta = param.Integer(default=60, bounds=(-90, 90), label="Theta (from z)")
    phi   = param.Integer(default=45, bounds=(0, 360), label="Phi (around z)")

    def __init__(self, **params):
        super().__init__(**params)
        self.plot_pane = pn.pane.Plotly(height=600, config={'responsive': True})
        self._update_plot()
        self.param.watch(self._update_plot, ['theta', 'phi'])
    def build_traces(self, r=4):
        theta_deg = self.theta
        phi_deg = self.phi
        v = spherical_to_cartesian(r, theta_deg, phi_deg)
        normal = np.array([0, 0, 1])
        proj = project_to_plane(v, normal)

        angle_rad = np.arccos(np.clip(np.linalg.norm(proj) / np.linalg.norm(v), -1, 1))
        angle_deg = np.round(np.degrees(angle_rad), 2)

        vector = go.Scatter3d(
            x=[0, v[0]], y=[0, v[1]], z=[0, v[2]],
            mode='lines+markers',
            line=dict(color='darkred', width=6),
            marker=dict(size=5, color='darkred'),
            name='Vector v'
        )

        leg1 = go.Scatter3d(
            x=[0, proj[0]], y=[0, proj[1]], z=[0, proj[2]],
            mode='lines',
            line=dict(color='black', width=4),
            name='Origin → proj(v)'
        )
        leg2 = go.Scatter3d(
            x=[proj[0], v[0]], y=[proj[1], v[1]], z=[proj[2], v[2]],
            mode='lines',
            line=dict(color='black', width=4),
            name='proj(v) → v'
        )

        xx, yy = np.meshgrid(np.linspace(-4, 4, 2), np.linspace(-4, 4, 2))
        zz = np.zeros_like(xx)
        plane = go.Surface(
            x=xx.tolist(), y=yy.tolist(), z=zz.tolist(),
            opacity=0.85,
            showscale=False,
            colorscale=[[0, '#989F7A'], [1, '#989F7A']],
            name='Plane'
        )
        return [plane, vector, leg1, leg2], angle_deg
    def _update_plot(self, *_):
        try:
            current_camera = self.plot_pane.object['layout']['scene']['camera']
        except Exception:
            current_camera = dict(eye=dict(x=2.0, y=0.65, z=0.30))

        traces, angle_deg = self.build_traces()
        fig = go.Figure(data=traces)
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-5, 5]),
                yaxis=dict(range=[-5, 5]),
                zaxis=dict(range=[-1, 5]),
                aspectmode='cube',
                camera=current_camera
            ),
            margin=dict(l=0, r=0, t=30, b=50),
            title=f"Vector Projection — Angle: {angle_deg:.2f}°",
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12))
        )
        self.plot_pane.object = fig
    def __panel__(self):
        return pn.Row(
            self.plot_pane,
            pn.Column(
                pn.Spacer(height=100),
                pn.widgets.Player.from_param(self.param.theta, width=250, interval=30, step=2, loop_policy='reflect'),
                pn.Spacer(height=10),
                pn.widgets.Player.from_param(self.param.phi, width=250, interval=30, step=3, loop_policy='reflect'),
            )
        )

In [11]:
#%output backend="plotly"
def create_plane(origin, v1, v2, size=2.0):
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)
    points = [origin + a * v1 + b * v2 for a in [-size, size] for b in [-size, size]]
    x = np.array([p[0] for p in points]).reshape(2, 2)
    y = np.array([p[1] for p in points]).reshape(2, 2)
    z = np.array([p[2] for p in points]).reshape(2, 2)
    return x, y, z
def plot_vector_pair(u_vec, v_vec, scale=1.6, u_color='darkred', v_color='blue', name_prefix='principal'):
    return [
        go.Scatter3d(x=[0, u_vec[0]*scale], y=[0, u_vec[1]*scale], z=[0, u_vec[2]*scale],
                     mode='lines', line=dict(color=u_color, width=6), name=f'{name_prefix}_u'),
        go.Scatter3d(x=[0, v_vec[0]*scale], y=[0, v_vec[1]*scale], z=[0, v_vec[2]*scale],
                     mode='lines', line=dict(color=v_color, width=6), name=f'{name_prefix}_v')
    ]
def angle_arc(a, b, origin=np.zeros(3), steps=30, scale=1.6, color='green', name='Angle Arc'):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    axis = np.cross(a, b)
    if np.linalg.norm(axis) < 1e-6:
        return None
    axis = axis / np.linalg.norm(axis)
    angle = np.arccos(np.clip(np.dot(a, b), -1.0, 1.0))
    pts = [R.from_rotvec(axis * t).apply(a) * scale for t in np.linspace(0, angle, steps)]
    pts = np.array(pts) + origin
    return go.Scatter3d(
        x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
        mode='lines', line=dict(color=color, width=4, dash='dot'),
        name=name
    )
def plot_intersection_line(u1, u2, v1, v2, scale=2.5):
    nU = np.cross(u1, u2)
    nV = np.cross(v1, v2)
    line_dir = np.cross(nU, nV)
    if np.linalg.norm(line_dir) < 1e-6:
        return None
    line_dir /= np.linalg.norm(line_dir)
    return go.Scatter3d(
        x=[-scale*line_dir[0], scale*line_dir[0]],
        y=[-scale*line_dir[1], scale*line_dir[1]],
        z=[-scale*line_dir[2], scale*line_dir[2]],
        mode='lines', line=dict(color='black', width=3, dash='dash'),
        name='Intersection Line'
    )

class PrincipalAnglePlayer(pn.viewable.Viewer):
    offset = param.Number(default=0.0, bounds=(-5.0, 5.0), label="Z Offset for Plane V")
    theta_1 = param.Integer(default=0, bounds=(-1800, 1800), label="theta_1 (10*degrees)")
    theta_2 = param.Integer(default=90, bounds=(-1800, 1800), label="theta_2 (10*degrees)")

    r     = param.Number(default=4.5, bounds=(1.0, 10.0), label="Camera Distance")
    theta = param.Number(default=45,  bounds=(0, 360), label="Azimuth (θ°)")
    phi   = param.Number(default=30,  bounds=(-90, 90), label="Elevation (φ°)")
    use_svd = param.Boolean(default=False, doc="Use SVD to compute principal vectors")

    def __init__(self, **params):
        super().__init__(**params)
        self.fig_pane = pn.pane.Plotly(height=650, config={'responsive': True})
        self.param.watch(self._update_data_only, ['theta_1', 'theta_2', 'offset', 'use_svd'])
        self.param.watch(self._update_camera_only, ['r', 'theta', 'phi'])
        self._current_camera = self._camera_eye()
        self._update_data_only()
    def _svd_from_planes(self, u1, u2, v1, v2):
        U, _ = np.linalg.qr(np.stack([u1, u2], axis=1))
        V, _ = np.linalg.qr(np.stack([v1, v2], axis=1))
        return np.linalg.svd(U.T @ V), U, V
    def compute_principal_vectors_and_angles(self, u1, u2, v1, v2):
        (W1, s, W2T), U, V = self._svd_from_planes(u1, u2, v1, v2)
        principal_U = U @ W1
        principal_V = V @ W2T.T
        angles = np.arccos(np.clip(s, -1, 1))
        return principal_U, principal_V, np.degrees(angles)
    def compute_geometric_principal_vectors(self, theta1_rad, theta2_rad):
        p1 = np.array([np.cos(theta1_rad), np.sin(theta1_rad), 0])
        perp = np.array([-np.sin(theta1_rad), np.cos(theta1_rad), 0])
        p2_v = np.cos(theta2_rad) * perp + np.sin(theta2_rad) * np.array([0, 0, 1])
        p2_v = p2_v / np.linalg.norm(p2_v)
        p2_u = perp / np.linalg.norm(perp)
        principal_U = np.stack([p1, p2_u], axis=1)
        principal_V = np.stack([p1, p2_v], axis=1)
        angles = np.arccos([np.clip(np.dot(p1, p1), -1, 1), np.clip(np.dot(p2_u, p2_v), -1, 1)])
        return principal_U, principal_V, np.degrees(angles)
    def _camera_eye(self):
        θ, φ = np.radians(self.theta), np.radians(self.phi)
        x = self.r * np.cos(θ) * np.cos(φ)
        y = self.r * np.sin(θ) * np.cos(φ)
        z = self.r * np.sin(φ)
        return dict(x=x, y=y, z=z)
    def _update_camera_only(self, *_):
        self._current_camera = self._camera_eye()
        if self.fig_pane.object:
            self.fig_pane.object.update_layout(scene_camera=dict(eye=self._current_camera))
    def _update_data_only(self, *_):
        θ1 = np.radians(self.theta_1 * 0.1)
        θ2 = np.radians(self.theta_2 * 0.1)

        u1 = np.array([1, 0, 0])
        u2 = np.array([0, 1, 0])
        origin = np.zeros(3)

        v1 = np.array([np.cos(θ1), np.sin(θ1), 0])
        v1_perp = np.array([-np.sin(θ1), np.cos(θ1), 0])
        v2 = np.cos(θ2) * v1_perp + np.sin(θ2) * np.array([0, 0, 1])

        xU, yU, zU = create_plane(origin, u1, u2)
        xV, yV, zV = create_plane(origin + np.array([0, 0, self.offset]), v1, v2)

        if self.use_svd:
            principal_U, principal_V, angles = self.compute_principal_vectors_and_angles(u1, u2, v1, v2)
        else:
            principal_U, principal_V, angles = self.compute_geometric_principal_vectors(θ1, θ2)

        traces = [
            go.Surface(x=xU, y=yU, z=zU, opacity=0.85, showscale=False,
                       colorscale=[[0, '#989F7A'], [1, '#989F7A']], name="Plane U"),
            go.Surface(x=xV, y=yV, z=zV, opacity=0.65, showscale=False,
                       colorscale=[[0, '#bbbbbb'], [1, '#bbbbbb']], name="Plane V")
        ]

        traces += plot_vector_pair(principal_U[:, 0], principal_V[:, 0], name_prefix='p1')
        traces += plot_vector_pair(principal_U[:, 1], principal_V[:, 1], name_prefix='p2')

        arc1 = angle_arc(principal_U[:,0], principal_V[:,0], origin=origin, color='black', name='θ₁ arc')
        arc2 = angle_arc(principal_U[:,1], principal_V[:,1], origin=origin, color='black', name='θ₂ arc')
        if arc1:
            traces.append(arc1)
        if arc2:
            traces.append(arc2)

        inter_line = plot_intersection_line(u1, u2, v1, v2)
        if inter_line:
            traces.append(inter_line)

        title_text = f"Principal Angles: θ₁ = {angles[0]:.2f}°, θ₂ = {angles[1]:.2f}°"

        fig = go.Figure(data=traces)
        fig.update_layout(
            title=title_text,
            scene=dict(
                aspectmode='cube',
                xaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='X'),
                yaxis=dict(tickmode='array', tickvals=[-2, -1, 0, 1, 2], range=[-2.5, 2.5], title='Y'),
                zaxis=dict(tickmode='array', tickvals=[-1, 0, 1, 2, 3], range=[-1.5, 3.0], title='Z'),
                camera=dict(eye=self._current_camera, projection=dict(type="orthographic"))
            ),
            legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5, font=dict(size=12)),
            margin=dict(l=0, r=0, t=50, b=0),
            showlegend=True
        )
        self.fig_pane.object = fig
    def __panel__(self):
        return pn.Row(
            pn.Column(
                pn.pane.Markdown("## Principal Angle Visualizer (Intersection Line + Arcs + Projections)"),
                self.fig_pane,
            ),
            pn.Column(
                "theta_1", pn.widgets.Player.from_param(self.param.theta_1, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                "theta_2", pn.widgets.Player.from_param(self.param.theta_2, width=250, interval=10, step=5, start=-1800, end=1800, loop_policy='reflect'),
                pn.Spacer(height=20),
                pn.widgets.Checkbox.from_param(self.param.use_svd, name="Use SVD"),
            ),
        )

# 
<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Hyperplanes and Principal Angles</strong>
</div>

# 1. Reminder: Basic Facts and Computation of Orthogonal Projections

To define the angles between two subspaces, we will make use of **orthogonal projections**.<br>
We begin by restating some basic facts.

**Reminder:**
* **Projection Matrices** $P$ are defined by the property $P^2 = P$.
* **Orthogonal Projection Matrices** further satisfy $P^T = P$.
* Given an **orthonormal basis** $B = \{\ q_1, q_2, \dots q_k\ \}$ for some subspace $S = \text{span}\ B,$<br>
an orthogonal projection matrix onto this subspace is given by $P = Q Q^T$, where $Q = \begin{pmatrix} q_1 & q_2 & \dots q_k \end{pmatrix}$

In [13]:
%%julia
Random.seed!(1212)
Q = Q_matrix(3,general=true)[:,1:2]
display(py_show("Example: Consider the plane defined by the basis vectors ", L"\;\;q_1 =", Q[:,1], L",\;\;q_2 =", Q[:, 2]))
P = Q*Q'
py_show( L"\therefore \;\; Q = ", Q, L"\qquad P = Q Q^T =", P )

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

* The **eigenvalues** of a projection matrix $P$ are 0 and 1
* The **trace** of a projection matrix is thus the dimension of the subspace it projects onto

In [14]:
%%julia
py_show(L"tr(P) = ", tr(P))

<IPython.core.display.Latex object>

____
#### Computation

Given a matrix $A$, we can compute the orthogonal projection matrix $P$ onto the column space $\mathscr{C}(A)$ in a number of ways. Let $\tilde{A}$ be a submatrix of $A$ obtained by removing free variable columns if any. Then

<div style="float:left;margin-left:1cm;">

| Decomposition method of $A\qquad$ | Orthogonal projection onto $\mathscr{C}(A)$ |
| ---------| ----------- |
| normal equation solution | $P = \tilde{A} \left( \tilde{A}^T \tilde{A} \right)^{-1} \tilde{A}^T$. |
| $Q R$ decomposition | $P = Q Q^T$. |
| reduced SVD $A = U_r \Sigma_r V_r^T$ | $P = U_r U_r^T$. |
</div>

In [92]:
%%julia
Random.seed!(1212)
A = gen_qr_problem_4(maxint=1)
A = A[:,1:3]//1
py_show("A=",A)

<IPython.core.display.Latex object>

In [93]:
%%julia
AtAinv = A'A \ 1I(3)//1
py_show( L"P = A \left(A^T A \right)^{-1} A^T = ", A*AtAinv*A')

<IPython.core.display.Latex object>

In [127]:
%%julia
W = gram_schmidt_w(Int.(A))
Q = normalize_columns( W )

py_show( L"\text{ Given } A = QR,\; P = Q Q^T =", Q*Q')

<IPython.core.display.Latex object>

In [128]:
%%julia
Uᵣ = svd(A).U
py_show( L"\text{ Given } A = U_r \Sigma_r V_r^T, \; P = U_r U_r^T =", 1//361, (x->Int.(round(N(x)))).(361*Uᵣ*Uᵣ'))

<IPython.core.display.Latex object>

# 2. Angles Between Vectors, and Between a Vector and a Subspace

## 2.1 Angle Between Two Vectors

The dot product and Cauchy's inequality resulted the definition of the angle between two non-zero vectors $u$ and $v$:

$\qquad \cos \theta = \frac{ u \cdot v}{ \Vert u \Vert \ \Vert v \Vert}$.

**Remark:**<br>
$\qquad$ we can readily define **the angle between a vector and a one dimensional subspace** (a line),<br>
$\qquad$ by using a basis vector for the line.

In [14]:
VectorAngleViewer().servable()

Row
    [0] HoloViews(Overlay, height=350, sizing_mode='fixed', width=400)
    [1] Spacer(width=30)
    [2] Column
        [0] Spacer(height=20)
        [1] LaTeX(str, renderer='katex', width=250)
        [2] Spacer(height=20)
        [3] FloatSlider(end=180, name='Angle (degrees)', value=45, width=400)

## 2.2 Angle between a Vector and a Subspace

Given a vector $v \in \mathbb{R}^n$ and a subspace $S \subseteq \mathbb{R}^n$,<br>
it is often useful to quantify how closely $v$ aligns with $S$.<br>
This leads to the generalization of the angle between two vectors to the angle between a vector and a subspace.

Let $P$ denote the orthogonal projection matrix of $v$ onto $S$. The angle $\theta$ between $v$ and the subspace $S$ is defined as:

$\qquad
\theta = \cos^{-1} \left( \frac{P\ v}{ \Vert v \Vert } \right).
$

This angle satisfies the following properties:
* $\theta = 0^\circ$ if and only if $v \in S$,
* $\theta = 90^\circ$ if and only if $v \perp S$,
* otherwise, $0^\circ < \theta < 90^\circ$.

**Geometric Interpretation:** The angle $\theta$ corresponds to the angle between $v$ and its projection onto $S$,<br> $\qquad$ i.e., the shortest angle from $v$ to any vector in $S$.

**Computation:** If $Q \in \mathbb{R}^{n \times k}$ is an orthonormal basis for $S$,<br>
$\qquad$ then the projection of $v$ onto $S$ is given by

$\qquad
P\ v = Q Q^T \ v,
$

and the angle becomes

$\qquad
\theta = \cos^{-1} \left( \frac{ Q Q^T \ v }{\Vert v \Vert} \right).
$

In [15]:
VectorProjectionDemo().servable()

Row
    [0] Plotly(Figure, config={'responsive': True}, height=600)
    [1] Column
        [0] Spacer(height=100)
        [1] Player(end=90, interval=30, loop_policy='reflect', name='Theta (from z)', start=-90, step=2, value=60, value_throttled=60, width=250)
        [2] Spacer(height=10)
        [3] Player(end=360, interval=30, loop_policy='reflect', name='Phi (around z)', step=3, value=45, value_throttled=45, width=250)

**Remark:** The demo also clearly shows that projection shortens the vector unless it is in the subspace.

# 3. Angles Between Subspaces

# FooX Products of Orthogonal Projection Matrices

Consider $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n,$ two subspaces of dimensions $k$ and $\ell$, with orthonormal bases
- $U \in \mathbb{R}^{n \times k}$ such that $U^T U = I_k,\quad$ i.e., the columns of $U$ are orthonormal
- $V \in \mathbb{R}^{n \times \ell}$ such that $V^T V = I_\ell,\quad$ i.e., the columns of $V$ are orthonormal

and let $P_u = U U^T$ and $P_v = V V^T$ be the orthogonal projection matrices onto $\mathcal{U}$ and $\mathcal{V}$ respectively.

____
**Projection Composition and Overlap:**

The matrix product $\quad P_u P_v = U U^T V V^T$

- Projects a vector onto $\mathcal{V}$, then onto $\mathcal{U}$
- $P_u P_v$ is not symmetric unless $𝑃_u = P_v$
- Note that **this product is not a projection operator in general** (since $\left( P_u P_v \right)^2 \ne \left( P_u P_v \right)$ in general.
- Encodes geometric information about the **intersection** and **relative orientation** of the subspaces


Consider the following **example in 2D:**
* Let $P_u$ be the orthogonal projection onto the x axis
* Let $P_v$ be the orthogonal projection onto the line $y = \tan(\theta)\ x$

Then $P_u P_v$ maps a vector to the projection onto the line, then onto the x axis:<br>
$\qquad$ The matrix $P_u P_v$ has eigenvalues $0, \cos^2(\theta)$.

In [114]:
%%julia
θ = symbols( "θ", real=true)
Pᵤ = [ 1 0; 0 0]
c = cos(θ); s = sin(θ)
Pᵥ = [c^2  s*c; s*c s^2]
display(py_show(L"P_u = ", Pᵤ, L",\quad P_v =", Pᵥ, L"\quad \therefore \;\; P_u P_v =", Pᵤ*Pᵥ))
py_show( L"\text{Since }  P_u P_v \text{ is upper diagonal, the eigenvalues are as stated}")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

____
**Geometric Interpretation**

- If $\mathcal{U} = \mathcal{V}$, then $P_u P_v = P_u = P_v$
- If $\mathcal{U} \perp \mathcal{V}$, then $P_u P_v = 0$
- In general, the **rank** of $P_u P_v$ reflects the number of shared directions
- The **eigenvectors** of $P_u P_v$ with eigenvalue 1 span the **intersection** $\mathcal{U} \cap \mathcal{V}$

## 1.3 Eigenvalues of Products of Orthogonal Projection Matrices

We will prove that **all eigenvalues $\lambda$ of $P_u P_v$ satisfy $0 \leq \lambda \leq 1$**.

Note that $P_u P_v$ is not symmetric in general, but we can study the related symmetric matrix:

$ \qquad
P_v P_u P_v = (P_v P_u)(P_v P_u)^T = V V^T U U^T V V^T
$

<div style="background-color:#F2F5A9;color:black;padding-bottom:0.05cm;">

**Thm:** Given orthogonal projection matrices $P_u$ and $P_v$. Then<br>
$\qquad P_u P_v$ is **positive semi-definite** with eigenvalues $0 \le \lambda \le 1$.
</div>

Let $(\lambda, x)$ be an eigenpair of $P_u P_v$ and note that if $P_v x = 0$ we have $\lambda=0$.

For $P_v x \ne 0$, we have<br>
$\quad \begin{align}
P_u P_v x = \lambda x
\quad & \Rightarrow \quad   x^T P_v P_u P_v x &=&\ \lambda\  x^T P_v x \\
& \Rightarrow \quad x^T (P_u P_v)^T P_u P_v x &=&\ \lambda\ x^T P^T_v P_v x \\
& \Rightarrow \quad \Vert P_u P_v x \Vert^2 &=&\ \lambda\ \Vert P_v x \Vert^2 \\
& \Rightarrow \quad \lambda  &=&\ \frac{\Vert P_u P_v x \Vert^2}{\Vert P_v x \Vert^2} \\
& \Rightarrow \quad \lambda  &\le&\ \frac{\Vert P_v x \Vert^2}{\Vert P_v x \Vert^2} = 1 \\
\end{align}$

where we have used $\Vert P_u y \Vert \le \Vert y \Vert$ for any $y$ since $P_u$ is a projection operator.

## 1.4 Interpretation of the Eigenvalues of $P_u P_v$

### 1.4.1 Geometric Meaning of Eigenvalues

Let $(\lambda, x)$ be an eigenpair of $P_u P_v$, i.e., $\;\;P_u P_v x = \lambda x$

$\qquad$ Thus $\lambda$ describes how much of $x$ survives when first projected onto $\mathcal{V}$ and then onto $\mathcal{U}$.

#### Case: Eigenvalue $\lambda = 1$

If $\lambda = 1$, then $\;P_u P_v x = x$

Therefore
- $P_v x \in \mathcal{U}$
- $x \in \mathcal{V}$ (since $P_v$ doesn't change $x$)
- $x \in \mathcal{U}$ (since $P_u$ doesn't change it either)

$\therefore$ **Eigenvalue 1 corresponds to directions in the intersection** $\mathcal{U} \cap \mathcal{V}$.<br>
$\qquad$ If $\dim(\mathcal{U} \cap \mathcal{V}) = d$, then $P_u P_v$ has **$\mathbf{d}$ eigenvalues equal to 1**.

#### Case: Eigenvalue $\lambda = 0$

If $\lambda = 0$, then $\;P_u P_v x = 0$

This can happen in several ways
1. **$x \in \mathcal{V}^\perp$**: so $P_v x = 0$
2. **$P_v x \in \mathcal{U}^\perp$**: so $P_u P_v x = 0$ even though $P_v x \ne 0$
3. **$x \in \mathcal{V}$ but orthogonal to $\mathcal{U}$**

In all cases, this means<br>
$\qquad$ **$x$ is annihilated by the chain of projections.**<br>
$\qquad$ That is, nothing survives both projections.

$\therefore$ **Eigenvalue 0 corresponds to directions in** $\mathcal{V}$ **that are orthogonal to** $\mathcal{U}$
**or vice versa.**

---

#### Summary: Eigenvalue Interpretation

<div style="width:50%;">

| Eigenvalue $\lambda$ | Meaning |
|----------------------|---------|
| $\lambda = 1$        | $x \in \mathcal{U} \cap \mathcal{V}$ (shared direction) |
| $\lambda = 0$        | either $x$ lies in $\mathcal{V}$ but orthogonal to $\mathcal{U}$, or $P_v x = 0$ |
| $0 < \lambda < 1$    | Partial overlap — $x$ projects nontrivially into both |
</div>

**Remark:**<br>
$\qquad$ The **multiplicity of $\lambda = 1$** equals $\dim(\mathcal{U} \cap \mathcal{V})$ <strong style="color:red;">

## 1.5 $P_u P_v$ and $P_v P_u$

We now examine the **composition** of $P_u$ and $P_v$

$\qquad
P_u P_v \ne P_v P_u\;\;
$ in general

I.e., **projection matrices do not commute**. The order of projection **matters**.

- $P_u P_v x$ means:
  first project $x$ onto $\mathcal{V}$, then project that result onto $\mathcal{U}$.
- $P_v P_u x$ means:
  first project $x$ onto $\mathcal{U}$, then project that result onto $\mathcal{V}$.

Unless one subspace is contained in the other, or they are aligned, the results will differ.

#### Shared Properties

Despite not being equal, they share some important features:

- Both $P_u P_v$ and $P_v P_u$ are **real** and **diagonalizable**
- They have the **same nonzero eigenvalues**
- Their eigenvalues lie in $[0, 1]$
- If $\theta_1, \dots, \theta_k$ are the **principal angles** between $\mathcal{U}$ and $\mathcal{V}$, then:

$\qquad
\text{Nonzero eigenvalues of } P_u P_v \text{ and } P_v P_u \text{ are } \cos^2(\theta_i)
$

This arises because they are related via SVD of $U^T V$.

#### Algebraic Explanation

Let $\;\;
U^T V = W_1 \Sigma W_2^T \quad \text{(SVD)}
$

Then
- $P_u P_v = U W_1 \Sigma W_2^T V^T$
- $P_v P_u = V W_2 \Sigma W_1^T U^T$

So they are **transposes** of each other under this decomposition.

Hence<br>
$\qquad
(P_u P_v)^T = P_v P_u
$

Thus, they are **adjoint to one another**, and their **singular values** (and hence nonzero eigenvalues) are the same.

#### Key Difference: Eigenvectors

Even though $P_u P_v$ and $P_v P_u$ share eigenvalues, they typically have **different eigenvectors**.

- Eigenvectors of $P_u P_v$ lie in $\mathbb{R}^n$ and are generally **not preserved** under switching the order
- Only in special cases (e.g., subspaces aligned, or 1D) do they commute

#### Special Case: When Do They Commute?

- If $\mathcal{U} \subseteq \mathcal{V}$ (or vice versa)
- If $\mathcal{U} = \mathcal{V}$
- If both projections are onto the same line or plane

Then:

$\qquad
P_u P_v = P_v P_u = P_u = P_v
$

### Example: Comparing $P_u P_v$ and $P_v P_u$ in $\mathbb{R}^2$

Let’s work with two 1D subspaces in $\; \mathbb{R}^2 \;$:

- Let $\; \mathcal{U} \;$ be the $x$-axis:<br>
  $\qquad
  u = \begin{bmatrix} 1 \\ 0 \end{bmatrix}
  \Rightarrow
  P_u = uu^T =
  \begin{bmatrix}
  1 & 0 \\
  0 & 0
  \end{bmatrix}
  $
- Let $\; \mathcal{V} \;$ be a line at angle $\; \theta \;$ from the $x$-axis:<br>
  $\qquad
  v = \begin{bmatrix} \cos\theta \\ \sin\theta \end{bmatrix}
  \Rightarrow
  P_v = vv^T =
  \begin{bmatrix}
  \cos^2\theta & \cos\theta \sin\theta \\
  \cos\theta \sin\theta & \sin^2\theta
  \end{bmatrix}
  $

1. **Compute** $\; P_u P_v \;$:

$\qquad
P_u P_v =
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
\cos\theta \sin\theta & \sin^2\theta
\end{bmatrix}
=
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
0 & 0
\end{bmatrix}
$

2. **Now compute** $\; P_v P_u \;$:

$\qquad
P_v P_u =
\begin{bmatrix}
\cos^2\theta & \cos\theta \sin\theta \\
\cos\theta \sin\theta & \sin^2\theta
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
=
\begin{bmatrix}
\cos^2\theta & 0 \\
\cos\theta \sin\theta & 0
\end{bmatrix}
$

We clearly see:

$\qquad
P_u P_v \ne P_v P_u
$

Each matrix projects into a **different subspace** after applying the other. The **resulting matrix** depends on the order.


#### But: Same Nonzero Eigenvalue

Let’s compute their eigenvalues.

Both have characteristic polynomial:

$\qquad
\det(A - \lambda I) = \lambda(\lambda - \cos^2\theta)
$

So the eigenvalues of both are:

$\qquad
\boxed{\{0, \cos^2\theta\}}
$

These correspond to:
- A **zero mode**: directions orthogonal to both
- A **principal direction**: aligned with both subspaces

#### Insight

This example shows:

- **Order matters** for composition of projections
- **But geometry still rules**: the amount of alignment (via principal angle) is shared

### Summary

<div style="float:left;margin-left:1cm;">

| Property              | $P_u P_v$ and $P_v P_u$ |
|-----------------------|-------------------------|
| Equal?                | No, not in general      |
| Share eigenvalues?    | Yes                     |
| Eigenvalue range      | $[0, 1]$                |
| Eigenvectors?         | Generally different     |
| Transpose of each other? | Yes: $(P_u P_v)^T = P_v P_u$ |
| Commute when?         | If subspaces align or contain one another |

</div>

## 1.6 Principal Angles

### 🔷 Principal Angles and the Matrix $P_u P_v$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be two subspaces of dimensions $k$ and $\ell$, respectively.

The **principal angles** $\theta_1, \theta_2, \dots, \theta_d$ between the subspaces are defined recursively as the **smallest angles between pairs of unit vectors**:

- $u_i \in \mathcal{U}$, $v_i \in \mathcal{V}$
- such that:
  - $\langle u_i, v_i \rangle = \cos(\theta_i)$
  - $u_i^T u_j = v_i^T v_j = 0$ for all $i \ne j$

Here, $d = \min(k, \ell)$ is the number of principal angles.

### 🎯 How They Arise from $P_u P_v$

Let:

- $P_u = UU^T$, where $U \in \mathbb{R}^{n \times k}$ is orthonormal basis for $\mathcal{U}$
- $P_v = VV^T$, with $V \in \mathbb{R}^{n \times \ell}$ for $\mathcal{V}$

Then:
- The matrix $U^T V \in \mathbb{R}^{k \times \ell}$ captures the **pairwise dot products** between basis vectors of the subspaces.
- Perform **SVD** on this matrix:<br>
  $\qquad
  U^T V = W_1 \Sigma W_2^T
  $
- The singular values $\sigma_i = \cos(\theta_i)$ are the **cosines of the principal angles**

### 🧠 Connection to $P_u P_v$

#### 🔍 Why Are the Eigenvalues of $P_U P_V$ the Squares of the Singular Values of $U^\top V$?

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be subspaces of dimensions $k$ and $\ell$, with orthonormal bases $U \in \mathbb{R}^{n \times k}$ and $V \in \mathbb{R}^{n \times \ell}$. Let:

- $P_U = UU^\top$ — projection onto $\mathcal{U}$
- $P_V = VV^\top$ — projection onto $\mathcal{V}$
- $M = U^\top V \in \mathbb{R}^{k \times \ell}$

---

#### Step 1: SVD of $U^\top V$

Compute the singular value decomposition (SVD):

$$
U^\top V = W_1 \Sigma W_2^\top
$$

Then:

- $\Sigma = \operatorname{diag}(\sigma_1, \dots, \sigma_d)$
- $\sigma_i = \cos(\theta_i)$ are the **cosines of the principal angles** between $\mathcal{U}$ and $\mathcal{V}$
- Define $A = UW_1$, $B = VW_2$, then:

$$
U^\top V = W_1 \Sigma W_2^\top \quad \Rightarrow \quad P_U P_V = A \Sigma B^\top
$$

---

#### Step 2: Action of $P_U P_V$

Let $z_i = B e_i = V W_2 e_i$. Then:

$$
P_U P_V z_i = A \Sigma e_i = \sigma_i A e_i
$$

Now apply $P_U P_V$ again:

$$
P_U P_V (A e_i) = A \Sigma^2 e_i = \sigma_i^2 A e_i
$$

So:
- $A e_i = U W_1 e_i$ is an eigenvector of $P_U P_V$
- With eigenvalue $\lambda_i = \sigma_i^2$

---

#### ✅ Conclusion

$$
\boxed{
\text{Eigenvalues of } P_U P_V \text{ are } \sigma_i^2 = \cos^2(\theta_i)
}
$$

That is:
- The **singular values** of $U^\top V$ are $\sigma_i = \cos(\theta_i)$
- The **eigenvalues** of $P_U P_V$ are the **squares** of these singular values

This directly links the **geometry of subspace alignment** (principal angles) to the **algebra of projection products**.


The matrix:

$\qquad
P_u P_v = U U^T V V^T
$

has **nonzero eigenvalues** equal to:

$\qquad
\cos^2(\theta_1), \cos^2(\theta_2), \dots, \cos^2(\theta_d)
$

So:

> **Principal angles govern the nonzero eigenvalues of the matrix product $P_u P_v$**

This reflects how "aligned" the subspaces are in each direction.

### 🧪 Summary of Geometric Meaning

<div style="width:50%;">

| Principal Angle $\theta_i$ | Interpretation |
|----------------------------|----------------|
| $\theta_i = 0$             | Shared direction — subspaces overlap |
| $0 < \theta_i < \frac{\pi}{2}$ | Partial alignment |
| $\theta_i = \frac{\pi}{2}$ | Orthogonal in that direction |
</div></div>
Then:

- $P_u P_v$ has **eigenvalue $1$** when $\theta_i = 0$
- $P_u P_v$ has **eigenvalue $0$** when $\theta_i = \frac{\pi}{2}$

### 📌 Conclusion

- Principal angles provide a **canonical, coordinate-free measure** of subspace alignment.
- The matrix $P_u P_v$ **encodes these alignments** via its eigenvalues $\lambda_i = \cos^2(\theta_i)$
- These angles are used in applications like:
  - Subspace comparison
  - Canonical correlation analysis
  - Dimensionality reduction & projections

# Y

## 2.3 Angle Between Subspaces

To quantify the relative orientation between two subspaces of equal dimension, we extend the idea of an angle between vectors.<br>
Instead of a single scalar, we measure how much the subspaces "tilt" toward or away from one another across all directions they span.

This leads to the concept of **principal angles**: a sequence of angles that capture the most aligned, then successively less aligned directions between the two subspaces.

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be two $k$-dimensional subspaces. The **principal angles** $\theta_1, \dots, \theta_k$ between them are recursively defined as the smallest angles between any unit vectors $u_i \in \mathcal{U}$ and $v_i \in \mathcal{V}$, subject to orthogonality constraints on previously chosen directions.

### Angle Between Two Subspaces and Principal Angles

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$ be two $k$-dimensional subspaces. To describe their relative orientation, we define the **principal angles** $\theta_1, \dots, \theta_k$ between them.

Assuming we have orthonormal bases:

- $U \in \mathbb{R}^{n \times k}$ for $\mathcal{U}$
- $V \in \mathbb{R}^{n \times k}$ for $\mathcal{V}$

We compute the matrix of inner products:

$$
M = U^T V,
$$

and perform a singular value decomposition (SVD):

$$
M = A \Sigma B^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k).
$$

The **principal angles** are:

$$
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k.
$$

---

### Example: Two 2D Subspaces in $\mathbb{R}^3$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^3$ be 2-dimensional subspaces with orthonormal bases:

- $\mathcal{U}$: $\{u_1, u_2\}$
- $\mathcal{V}$: $\{v_1, v_2\}$

The principal angles are then directly computed from dot products:

$$
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right), \qquad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right).
$$

These angles quantify how closely the two planes align, ranging from $0^\circ$ (perfect alignment) to $90^\circ$ (orthogonal subspaces).

#### Formal Computation

Assume we have orthonormal bases:

- $U \in \mathbb{R}^{n \times k}$ for $\mathcal{U}$
- $V \in \mathbb{R}^{n \times k}$ for $\mathcal{V}$

1. Compute the matrix of inner products:

$$
M = U^T V
$$

2. Perform a singular value decomposition (SVD):

$$
M = A \Sigma B^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k)
$$

3. The **principal angles** are given by:

$$
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k
$$

Each angle lies in the range $[0^\circ, 90^\circ]$:
- $\theta_i = 0^\circ$ means the subspaces overlap in that direction
- $\theta_i = 90^\circ$ means they are orthogonal in that direction

---

#### Example: Two 2D Subspaces in $\mathbb{R}^3$

Let $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^3$ be two 2D subspaces with orthonormal bases:

- $\mathcal{U}$: $\{u_1, u_2\}$
- $\mathcal{V}$: $\{v_1, v_2\}$

Then:

$$
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right), \qquad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right)
$$

These two angles describe the full geometric relationship between the planes.

____

When comparing two subspaces, especially of the same dimension, the notion of an angle generalizes beyond the case of a single vector and a plane.

Given two $k$-dimensional subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$, the angle between them is not a single scalar, but a set of angles known as **principal angles**.

Intuitively, the smallest principal angle measures how close the two subspaces are to overlapping along a common direction.
* If two subspaces share a common vector direction, the smallest angle will be $0^\circ$.
* If they are orthogonal, the largest principal angle will be $90^\circ$.

The first principal angle $\theta_1$ is defined as the smallest angle between any two unit vectors $u \in \mathcal{U}$ and $v \in \mathcal{V}$:

$\qquad
\cos(\theta_1) = \max_{u \in \mathcal{U}, \ v \in \mathcal{V}} \langle u, v \rangle \quad \text{subject to} \quad \|u\| = \|v\| = 1.
$

Subsequent principal angles are defined recursively, constraining the directions to be orthogonal to all previously chosen ones.<br>
These angles give a complete geometric relationship between the two subspaces.

### 2.3.1 Computing Principal Angles Between Subspaces

Let $\mathcal{U}$ and $\mathcal{V}$ be two 2-dimensional subspaces of $\mathbb{R}^3$.

We define **orthonormal bases** for each subspace:
- For $\mathcal{U}$: $\{u_1, u_2\}$
- For $\mathcal{V}$: $\{v_1, v_2\}$

The **principal angles** $\theta_1$ and $\theta_2$ between the subspaces are defined recursively as the smallest angles between pairs of unit vectors:

- $\theta_1$ is the smallest angle between any $u \in \mathcal{U}$ and $v \in \mathcal{V}$
- $\theta_2$ is the next smallest angle between directions **orthogonal to the first pair**

In this 2D case with known bases, the principal angles are computed directly as:

$\qquad
\cos(\theta_1) = \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|}, \quad
\theta_1 = \cos^{-1} \left( \frac{\langle u_1, v_1 \rangle}{\|u_1\| \|v_1\|} \right)
$

$\qquad
\cos(\theta_2) = \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|}, \quad
\theta_2 = \cos^{-1} \left( \frac{\langle u_2, v_2 \rangle}{\|u_2\| \|v_2\|} \right)
$

This approach uses the fact that:
- $u_1, u_2$ form an orthonormal basis for $\mathcal{U}$
- $v_1, v_2$ are obtained by rotating and tilting the basis vectors of $\mathcal{U}$ into $\mathcal{V}$

The computed angles $\theta_1$ and $\theta_2$ describe the geometric alignment between the subspaces.

### 2.3.2 Interactive Figure of Two Planes in 3D

### 2.3.4 Computing Principal Angles from Arbitrary Bases

Let $\mathcal{U}, \mathcal{V} \subset \mathbb{R}^n$ be two $k$-dimensional subspaces, each defined by a basis:

- $A \in \mathbb{R}^{n \times k}$: a matrix whose columns form a basis for $\mathcal{U}$
- $B \in \mathbb{R}^{n \times k}$: a matrix whose columns form a basis for $\mathcal{V}$

These bases do **not** need to be orthonormal.

#### Step 1: Orthonormalize the Bases

Begin by orthonormalizing the columns of $A$ and $B$ using QR decomposition:

$\qquad
A = Q_U R_U, \quad B = Q_V R_V
$

where $Q_U, Q_V \in \mathbb{R}^{n \times k}$ have orthonormal columns that span $\mathcal{U}$ and $\mathcal{V}$ respectively.

#### Step 2: Compute the Matrix of Inner Products

Compute the matrix of cosines between the orthonormal bases:

$\qquad
M = Q_U^T Q_V \in \mathbb{R}^{k \times k}
$

#### Step 3: Singular Value Decomposition

The **principal angles** are obtained from the singular values of $M$:

$\qquad
M = U \Sigma V^T, \quad \Sigma = \mathrm{diag}(\sigma_1, \dots, \sigma_k)
$

Then the principal angles are:

$\qquad
\theta_i = \cos^{-1}(\sigma_i), \quad \text{for } i = 1, \dots, k
$

Each $\theta_i \in [0^\circ, 90^\circ]$ measures how aligned the $i$-th principal direction of $\mathcal{U}$ is with that of $\mathcal{V}$.

____
**Remark:** This procedure generalizes the concept of the angle between two vectors to higher-dimensional subspaces.

## 2.4 Projection Matrices and Subspace Intersections

The intersection of two subspaces can be investigated using their projection matrices:
- Let `P = UUᵀ` and `Q = VVᵀ` be orthogonal projections.
- Then the intersection of the subspaces corresponds to the **invariant subspace** of `PQ` or `QP`.

The angle between subspaces is also encoded in the eigenvalues of `PQ`:
- If `PQ` has an eigenvalue of `1`, it corresponds to a shared direction.
- Singular values of `UᵀV` relate to `cos(θᵢ)` for principal angles.

This builds geometric insight into the SVD structure used in computing principal directions.

The concept of principal angles offers a natural way to compare subspaces of equal dimension. For two $k$-dimensional subspaces $\mathcal{U}, \mathcal{V} \subseteq \mathbb{R}^n$, there are $k$ principal angles $\theta_1, \dots, \theta_k$ defined recursively.

Each angle $\theta_i$ captures the cosine of the largest possible inner product between unit vectors $u_i \in \mathcal{U}$ and $v_i \in \mathcal{V}$, under the constraint that $u_i$ and $v_i$ are orthogonal to the previously selected vectors:

$$
\cos(\theta_i) = \max_{u \in \mathcal{U}, \ v \in \mathcal{V}} \langle u, v \rangle, \quad \text{subject to} \quad \|u\| = \|v\| = 1, \ u \perp u_j, \ v \perp v_j \ \text{for } j < i.
$$

The collection of angles describes how the subspaces align:
- If all $\theta_i = 0$, the subspaces are identical.
- If all $\theta_i = 90^\circ$, the subspaces are orthogonal.
- Intermediate values reflect partial alignment.

These angles are closely related to the singular values of $U^T V$, where $U$ and $V$ are orthonormal bases for $\mathcal{U}$ and $\mathcal{V}$ respectively.

## Principal Angle Visualizer

The following demo illustrates principal directions and angles for two 3D planes intersecting in a line.

- **Plane U (dark)**: fixed in the XY plane.
- **Plane V (light)**: defined by azimuth (`θ₁`) and elevation (`θ₂`).
  - `θ₁`: rotates Plane V about Z-axis (in-plane direction).
  - `θ₂`: tilts Plane V upward (out-of-plane direction).
- **Principal Directions**:
  - First pair (`red`): aligned with line of intersection.
  - Second pair (`green`): orthogonal in-plane directions.
- **Arcs** (`θ₁ arc`, `θ₂ arc`): show angle between principal directions.
- **Intersection Line**: dashed black line — true geometric overlap.

In [16]:
PrincipalAnglePlayer().servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure, config={'responsive': True}, height=650)
    [1] Column
        [0] Markdown(str)
        [1] Player(end=1800, interval=10, loop_policy='reflect', name='theta_1 (10*degrees)', start=-1800, step=5, width=250)
        [2] Markdown(str)
        [3] Player(end=1800, interval=10, loop_policy='reflect', name='theta_2 (10*degrees)', start=-1800, step=5, value=90, value_throttled=90, width=250)
        [4] Spacer(height=20)
        [5] Checkbox(name='Use SVD')